<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/LakeTimeSeriesv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trying a change

#**Description:** 
This script ingests a shapefile of individual waterbodies and exports a csv of water, cloudy, and total pixels for each day of valid observations. Observations through time of individual waterbodies are merged outside of Earth Engine (script under development).

#**Directions:** 
- Run the 'Setup' chunk and to authenticate your EE account with google colab. Currently the google drive and colab user authentication are commented out, but for development purposes sometimes it's helpful to include those.
- Set your input and export variables and run the chunk. 
- Run the 'Functions' chunk. Note: you can collapse chunks for ease of use.
- Run the 'Main' chunk.
- Export progress can be monitored at code.earthengine.google.com under the 'Tasks' panel.

#**Potential Errors:**
Certain aspects of this workflow may need some tweaking in order to adapt to different analyses and starting shapefiles:
- Currently set to run within one UTM zone.
- Some of the waterbody properties depend on attributes associated with the imported shapefile, such as centroid, count, and ID. The 'lakeProps' function that combines all of the waterbody property extraction functions will likely fail with some of these attributes missing. In the future I can put in some try/except statements to avoid errors, but for now we likely have to alter the property extraction functions for specific use cases.
- Cloudy image filtering currently occurs based on the cloudiness of the entire ROI. This can easily be adjusted to just include cloudiness over the lakes, but filtering per lake will be difficult. One option is to just not do any filtering because cloudy pixels are counted in the exported time series.

#### Setup

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
#from google.colab import auth 
#auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize() # Initialize the library.

# Google Drive setup (if needed)
#from google.colab import drive
#drive.mount('/content/drive')

# Some common imports
from IPython.display import Image
import folium
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap']) 

import geemap

#### Input and Output Settings

In [ ]:
## ***INPUTS***

# Time Period Dates
start = ee.Date('2020-07-1')
finish = ee.Date('2020-07-10')
# Region of Interest
xmin = -151.730779
xmax = -148.527340
ymin =  68.648661
ymax =  69.756928
roi = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
# UTM zone
epsg = 'EPSG:32606'
# Lake Shapefile
lakes = ee.FeatureCollection('projects/toolik/assets/ToolikLakes1')
# Minimum Pixel Count for a lake 
minPix = 10 
# Image scale
pixScale = 10

# ***EXPORTS***

# Export Properties
exportSelectors = ['id', 'waterPixels', 'allPixels', 'clearPixels', 'cloudPixels', 'count', 'centroid', 'ratio'] # These will likely need to be changed to reflect your input shapefile. Specifically, 'count', 'centroid', and 'ratio' are all attributes from the imported EE asset. The others can stay the same.
# ROI Description
roiLabel = 'Toolik'
# Export Folder
exportFolder = 'tooliktimeseries'

#### Functions

In [ ]:
## ***IMAGE PRE-PROCESSING METHODS***

# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

def clip2lakes(image):
  '''Clips an image based on the lake boundaries'''
  return image.clip(lakes)

# Get percentile cover   
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actPixels = ee.Number(image.updateMask(image.select('clear_mask')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      geometry = roi,
      maxPixels=1e12,
      ).values().get(0))
  # calculate the perc of cover OF CLEAR PIXELS 
  percCover = actPixels.divide(totPixels).multiply(100).round()
  # number as output
  return image.set('percCover', percCover,'actPixels',actPixels)
  
# Mosaic images by date, orbit, - basically combines images together that were taken on the same day 
def mosaicBy(imcol):
  '''Takes an image collection (imcol) and creates a mosaic for each day
  Returns: An image collection of daily mosaics'''
  #return the collection as a list of images (not an image collection)
  imlist = imcol.toList(imcol.size())
  # Get all the dates as list
  def imdate(im):
    date = ee.Image(im).date().format("YYYY-MM-dd")
    return date
  all_dates = imlist.map(imdate)
  # get all orbits as list
  def orbitId(im):
    orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
    return orb
  all_orbits = imlist.map(orbitId)
  # get all spacecraft names as list
  def spacecraft(im):
    return ee.Image(im).get('SPACECRAFT_NAME')
  all_spNames = imlist.map(spacecraft)
  # this puts dates, orbits and names into a nested list
  concat_all = all_dates.zip(all_orbits).zip(all_spNames);
  # here we unnest the list with flatten, and then concatenate the list elements with " "
  def concat(el):
    return ee.List(el).flatten().join(" ")
  concat_all = concat_all.map(concat)
  # here, just get distinct combintations of date, orbit and name
  concat_unique = concat_all.distinct()
  # mosaic
  def mosaicIms(d):
    d1 = ee.String(d).split(" ")
    date1 = ee.Date(d1.get(0))
    orbit = ee.Number.parse(d1.get(1)).toInt()
    spName = ee.String(d1.get(2))
    im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
    return im.set(
        "system:time_start", date1.millis(),
        "system:date", date1.format("YYYY-MM-dd"),
        "system:id", d1)
  mosaic_imlist = concat_unique.map(mosaicIms)
  return ee.ImageCollection(mosaic_imlist)

def reprojectMosaic(image):
  '''Reproject to UTM. A future function should take the image location and return
  the UTM zone. For now, I'm manually entering the EPSG code.'''
  image_projected = image.reproject(epsg)
  return image_projected

###########################################################################
## ***WATER CLASSIFICATION METHODS***

# Define NDWI image
def ndwi(image):
  '''Adds an NDWI band to the input image'''
  return image.normalizedDifference(['B3', 'B8']).rename('NDWI').multiply(1000)

# Basic ndwi classification  
def ndwi_classify(image):
  '''Creates a binary image based on an NDWI threshold of 0'''
  ndwimask = image.select('NDWI')
  water = ndwimask.gte(0)
  land = ndwimask.lt(0)
  return(water)

# OTSU thresholding from histogram
def otsu(histogram):
  '''Returns the NDWI threshold for binary water classification'''
  counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
  means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  indices = ee.List.sequence(1, size)
  def func_xxx(i):
    '''Compute between sum of squares, where each mean partitions the data.'''
    aCounts = counts.slice(0, 0, i)
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts) \
        .reduce(ee.Reducer.sum(), [0]).get([0]) \
        .divide(aCount)
    bCount = total.subtract(aCount)
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(
           bCount.multiply(bMean.subtract(mean).pow(2)))
  bss = indices.map(func_xxx)
  # Return the mean value corresponding to the maximum BSS.
  return means.sort(bss).get([-1])

# OTSU thresholding for an image
def otsu_thresh(water_image):
  '''Calculate NDWI and create histogram. Return the OTSU threshold.'''
  NDWI = ndwi(water_image).select('NDWI').updateMask(water_image.select('clear_mask'))
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  return otsu(histogram.get('NDWI_histogram'))

# Classify an image using OTSU threshold.
def otsu_classify(water_image):
  '''(1) Calculate NDWI and create histogram. (2) Calculate NDWI threshold for 
  binary classification using OTSU method. (3) Classify image and add layer to input image.
  '''
  NDWI = ndwi(water_image).select('NDWI')
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  threshold = otsu(histogram.get('NDWI_histogram'))
  otsu_classed = NDWI.gt(ee.Number(threshold)).And(water_image.select('B8').lt(2000)).rename('otsu_classed')
  return water_image.addBands([otsu_classed])

def adaptive_thresholding(water_image):
  '''Takes an image clipped to lakes and returns the water mask'''
  NDWI = ndwi(water_image).select('NDWI')#.updateMask(water_image.select('clear_mask')) # get NDWI **TURNED OFF CLOUD MASK, SHOULD THIS STAY OFF?**
  threshold = ee.Number(otsu_thresh(water_image)) 
  threshold = threshold.divide(10).round().multiply(10)
  # get fixed histogram
  histo = NDWI.reduceRegion(
      geometry = roi,
      reducer = ee.Reducer.fixedHistogram(-1000, 1000, 200),
      scale = pixScale, # This was 30, keep at 10!?!?
      maxPixels = 1e12
  )
  hist = ee.Array(histo.get('NDWI'))
  counts = hist.cut([-1,1])
  buckets = hist.cut([-1,0])
  #find split points from otsu threshold
  threshold = ee.Array([threshold]).toList()
  buckets_list = buckets.toList()
  split = buckets_list.indexOf(threshold)
  # split into land and water slices
  land_slice = counts.slice(0,0,split)
  water_slice = counts.slice(0,split.add(1),-1)
  # find max of land and water slices
  land_max = land_slice.reduce(ee.Reducer.max(),[0])
  water_max = water_slice.reduce(ee.Reducer.max(),[0])
  land_max = land_max.toList().get(0)
  water_max = water_max.toList().get(0)
  land_max = ee.List(land_max).getNumber(0)
  water_max = ee.List(water_max).getNumber(0)
  #find difference between land, water and otsu val
  counts_list = counts.toList()
  otsu_val = ee.Number(counts_list.get(split))
  otsu_val = ee.List(otsu_val).getNumber(0)
  land_prom = ee.Number(land_max).subtract(otsu_val)
  water_prom = ee.Number(water_max).subtract(otsu_val)
  #find land and water buckets corresponding to 0.9 times the prominence
  land_thresh = ee.Number(land_max).subtract((land_prom).multiply(ee.Number(0.9)))
  water_thresh = ee.Number(water_max).subtract((water_prom).multiply(ee.Number(0.9)))
  land_max_ind = land_slice.argmax().get(0)
  water_max_ind = water_slice.argmax().get(0)
  li = ee.Number(land_max_ind).subtract(1)
  li = li.max(ee.Number(1))
  wi = ee.Number(water_max_ind).add(1)
  wi = wi.min(ee.Number(199))
  land_slice2 = land_slice.slice(0,li,-1).subtract(land_thresh)
  water_slice2 = water_slice.slice(0,0,wi).subtract(water_thresh)
  land_slice2  = land_slice2.abs().multiply(-1)
  water_slice2 = water_slice2.abs().multiply(-1)
  land_index = ee.Number(land_slice2.argmax().get(0)).add(land_max_ind)
  water_index = ee.Number(water_slice2.argmax().get(0)).add(split)
  land_level = ee.Number(buckets_list.get(land_index))
  water_level = ee.Number(buckets_list.get(water_index))
  land_level = ee.Number(ee.List(land_level).get(0)).add(5)
  water_level = ee.Number(ee.List(water_level).get(0)).add(5)
  #calculate water fraction and classify
  water_fraction = (NDWI.subtract(land_level)).divide(water_level.subtract(land_level)).multiply(100).rename('water_fraction')
  #water_fraction = conditional(water_fraction) #sets values less than 0 to 0 and greater than 100 to 100
  water_75 = water_fraction.gte(75).rename('water_75'); #note, this is a non-binary classification, so we use 75% water as "water"
  all_mask = water_image.select('B2').gt(5).rename('all_mask')
  cloud_mask_ed = water_image.select('clear_mask').neq(1).rename('cloud_mask_ed')
  return water_image.addBands([water_fraction,water_75,NDWI,cloud_mask_ed])

###############################################################################
## ***PROPERTY EXTRACTION METHODS***
def sumWater(lake):
  '''sums the water pixels within a buffered lake polygon and adds the result to the feature'''
  watersum = lakeIm.select('water_75').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('water_75')
  return watersum

def sumClear(lake):
  '''sums the number of clear pixels within a buffered lake polygon'''
  clearsum = lakeIm.select('clear_mask').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('clear_mask')
  return clearsum

def sumClouds(lake):
  '''sums the number of clear pixels within a buffered lake polygon'''
  cloudsum = lakeIm.select('cloud_mask_ed').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('cloud_mask_ed')
  return cloudsum

def sumAll(lake):
  '''sums the number of pixels within a buffered lake polygon'''
  all_mask = lakeIm.select('B2').gt(5).rename('all_mask')
  allsum = all_mask.reduceRegion(
      reducer=ee.Reducer.count(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
      ).get('all_mask')
  return allsum

def troid(lake):
  center = ee.Array(lake.centroid().geometry().coordinates())
  return center

def getID(lake):
  id = ee.Number(lake.id())
  return id

def getCount(lake):
  count = ee.Number(lake.get('count'))
  return count

def getLabel(lake):
  label = ee.String(lake.get('label'))
  return label

def lakeProps(lake):
  water = sumWater(lake)
  clear = sumClear(lake)
  clouds = sumClouds(lake)
  all = sumAll(lake)
  centroid = troid(lake)
  id = getID(lake)
  count = getCount(lake)
  label = getLabel(lake)
  return ee.Feature(None, {'id': id, 'label':label, 'waterPixels': water, 'clearPixels': clear, 'cloudPixels': clouds, 'allPixels': all, 'centroid': centroid,'count': count})

###############################################################################
## ***EXPORT METHODS***
def export_lakes(collection, description, fileNamePrefix, fileFormat, folder, selectors):
  '''Export a feature collection of lake properties to google drive for a given day.'''
  task = ee.batch.Export.table.toDrive(**{
    'collection': collection,
    'description': description, 
    'fileNamePrefix': fileNamePrefix,
    'fileFormat': fileFormat,
    'folder': folder,
    'selectors': selectors
  })
  task.start()

#### Main

In [ ]:
###############################################################################
## ***Data Pre-Processing***

# Filter lakes
lakes = lakes.filter(ee.Filter.gt('count', minPix))

# Get Images, mosaic, clip to ROI, filter by coverage, and clip to lake mask
images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
images_all = mosaicBy(images) # Mosaic images
images_all = images_all.map(maskS2clouds) # Create cloud/clear masks
# Filter by percentile cover
image_mask = images.select('B2').mean().clip(roi).gte(0) #First, calculate total number of pixels
totPixels = ee.Number(image_mask.reduceRegion(
    reducer = ee.Reducer.count(),
    scale = 100,
    geometry = roi,
    maxPixels = 1e12
    ).values().get(0))
images_all = images_all.map(clip_image) # Clip to roi
images_all = images_all.map(getCover) # Add percent cover as an image property
images = images_all.filterMetadata('percCover','greater_than',50) # remove images covering less than 50% of the ROI)
lakeimages = images.map(clip2lakes) # Clip images to buffered lake mask
dates = lakeimages.aggregate_array('system:date').getInfo() # get a non-EE list of dates so it is iterable.

###############################################################################
## ***WATER CLASSIFICATION***
lakeimages = lakeimages.map(adaptive_thresholding)

###############################################################################
## ***ITERATE THROUGH DAYS AND EXPORT***
for i, date in enumerate(dates):
  # Get lake properties
  eedate = ee.Date(date)
  lakeIm = lakeimages.filterDate(eedate).first()
  lakes2 = lakes.map(lakeProps)
  # Export
  exportDate = date.replace('-', '_')
  description = roiLabel +'_'+ exportDate
  fileformat = 'CSV'
  export_lakes(lakes2, description, description, fileformat, exportFolder, exportSelectors)